In [1]:
import numpy as np
import tensorflow as tf

2023-09-25 00:14:01.366608: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 00:14:01.871714: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_samples = 1_000
n_features = 7
x = np.random.randn(n_samples, n_features)
names = np.random.randint(0, 100, size=n_samples).astype(str)
x, names

(array([[-1.02330989,  1.10112117,  0.26816211, ..., -1.93717991,
         -1.23120198,  1.75488616],
        [-0.53454485,  1.09275624, -0.69104461, ..., -0.69627258,
          0.19419093, -1.66526972],
        [-0.80634423, -0.21945347, -0.65996497, ...,  0.0094244 ,
          0.41984597,  1.61488729],
        ...,
        [-0.84350663, -0.42786854,  0.73615638, ...,  0.50393727,
         -0.67110885,  1.20727151],
        [ 0.87365795,  0.63888648,  1.63800232, ..., -1.47422034,
          0.63167284,  1.18165092],
        [ 1.24367518, -0.16088102, -0.54212906, ...,  0.67514079,
         -0.21209837,  0.97833255]]),
 array(['49', '45', '18', '68', '36', '21', '98', '60', '31', '79', '93',
        '65', '32', '50', '96', '65', '65', '25', '65', '61', '62', '12',
        '38', '74', '66', '11', '93', '18', '37', '94', '44', '24', '30',
        '54', '57', '14', '63', '66', '13', '35', '14', '43', '87', '13',
        '80', '61', '7', '5', '63', '39', '99', '81', '83', '20', '97',
     

In [13]:
example_path = 'example.tfrecords'
# Write the records to a file.
with tf.io.TFRecordWriter(example_path) as file_writer:
    for i in range(n_samples):
        record_bytes = tf.train.Example(
            features=tf.train.Features(feature={
                "f": tf.train.Feature(float_list=tf.train.FloatList(value=x[i])),
                "name": tf.train.Feature(bytes_list=tf.train.BytesList(value=[names[i].encode('UTF-8')])),
            })).SerializeToString()
        file_writer.write(record_bytes)

In [18]:
def tfrecord_decoder(record_bytes):
    return tf.io.parse_single_example(
        record_bytes,
        
        # schema
        {'f': tf.io.FixedLenFeature([n_features], dtype=tf.float32),
         'name': tf.io.FixedLenFeature([], dtype=tf.string)}
    )

In [19]:
for batch in tf.data.TFRecordDataset([example_path]).map(tfrecord_decoder):
    print(batch)
    break

{'f': <tf.Tensor: shape=(7,), dtype=float32, numpy=
array([-1.02331   ,  1.1011212 ,  0.2681621 ,  0.37342593, -1.9371799 ,
       -1.231202  ,  1.7548862 ], dtype=float32)>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'49'>}
